In [458]:
# BPE tokenization
text = "students play soccer cat is playing with students"

In [459]:
wordList = list(set(text.split()))
print(f'word list: {wordList}')

word list: ['play', 'cat', 'is', 'soccer', 'playing', 'with', 'students']


In [460]:
corpus = {}
for word in text.split():
    cleaned_word = word.strip('.,!?').lower()
    if cleaned_word in corpus:
        corpus[cleaned_word] += 1
    else:
        corpus[cleaned_word] = 1
print(f'init corpus: {corpus}')

splits = {}
vocab = set() 
for word in wordList:
    _split = [word[0]]
    for i in range(1, len(word)):
        _split.append('##' +word[i])
        vocab.add('##' + word[i])
    splits[word] = _split
print(f'init splits: {splits}')
print(f'init vocab: {vocab}')

init corpus: {'students': 2, 'play': 1, 'soccer': 1, 'cat': 1, 'is': 1, 'playing': 1, 'with': 1}
init splits: {'play': ['p', '##l', '##a', '##y'], 'cat': ['c', '##a', '##t'], 'is': ['i', '##s'], 'soccer': ['s', '##o', '##c', '##c', '##e', '##r'], 'playing': ['p', '##l', '##a', '##y', '##i', '##n', '##g'], 'with': ['w', '##i', '##t', '##h'], 'students': ['s', '##t', '##u', '##d', '##e', '##n', '##t', '##s']}
init vocab: {'##o', '##l', '##c', '##t', '##i', '##g', '##s', '##e', '##h', '##d', '##y', '##u', '##a', '##n', '##r'}


In [461]:
print(splits.values())

dict_values([['p', '##l', '##a', '##y'], ['c', '##a', '##t'], ['i', '##s'], ['s', '##o', '##c', '##c', '##e', '##r'], ['p', '##l', '##a', '##y', '##i', '##n', '##g'], ['w', '##i', '##t', '##h'], ['s', '##t', '##u', '##d', '##e', '##n', '##t', '##s']])


In [462]:
def Score(splits: list[list], key1, key2):
    _countPair = 0
    _count1 = 0
    _count2 = 0
    for split in splits:
        index = 0
        for index in range(len(split) - 1):
            if split[index] == key1 and split[index+1] == key2:
                _countPair += 1
            if split[index] == key1:
                _count1 += 1
            if split[index] == key2:
                _count2 += 1
        _count1 += (split[-1] == key1)
        _count2 += (split[-1] == key2)
    
    return _countPair / (_count1 * _count2)

Score(list(splits.values()), 'p', '##l')

0.5

In [463]:
def cleanHash(text):
    pos = 0
    while text[pos] == '#':
        pos += 1
    return text[pos:]
        
def combine_elements(lst, key1, key2):
    result = []
    i = 0
    
    while i < len(lst):
        if i < len(lst) - 1 and lst[i] == key1 and lst[i+1] == key2:
            result.append(key1 + cleanHash(key2))
            i += 2
        else:
            result.append(lst[i])
            i += 1
    
    return result


In [464]:
# training phase
train_iter = 25
while len(vocab) < train_iter:
    dic = {}
    top_key1, top_key2, top_val = '','', 0

    # find the pair with highest score
    for split in splits.values():
        for index in range(0, len(split) - 1):
            _score = Score(list(splits.values()), split[index], split[index+1])
            if  _score > top_val:
                top_key1 = split[index]
                top_key2 = split[index+1]
                top_val = _score

    vocab.add(top_key1 + cleanHash(top_key2))
    print(f'added: {top_key1}-{top_key2} with score: {top_val}')

    # update splits
    for w, split in splits.items():
        splits[w] = combine_elements(split, top_key1, top_key2)
    print(f'updated splits {splits}')

added: ##u-##d with score: 1.0
updated splits {'play': ['p', '##l', '##a', '##y'], 'cat': ['c', '##a', '##t'], 'is': ['i', '##s'], 'soccer': ['s', '##o', '##c', '##c', '##e', '##r'], 'playing': ['p', '##l', '##a', '##y', '##i', '##n', '##g'], 'with': ['w', '##i', '##t', '##h'], 'students': ['s', '##t', '##ud', '##e', '##n', '##t', '##s']}
added: p-##l with score: 0.5
updated splits {'play': ['pl', '##a', '##y'], 'cat': ['c', '##a', '##t'], 'is': ['i', '##s'], 'soccer': ['s', '##o', '##c', '##c', '##e', '##r'], 'playing': ['pl', '##a', '##y', '##i', '##n', '##g'], 'with': ['w', '##i', '##t', '##h'], 'students': ['s', '##t', '##ud', '##e', '##n', '##t', '##s']}
added: i-##s with score: 0.5
updated splits {'play': ['pl', '##a', '##y'], 'cat': ['c', '##a', '##t'], 'is': ['is'], 'soccer': ['s', '##o', '##c', '##c', '##e', '##r'], 'playing': ['pl', '##a', '##y', '##i', '##n', '##g'], 'with': ['w', '##i', '##t', '##h'], 'students': ['s', '##t', '##ud', '##e', '##n', '##t', '##s']}
added: s-##

In [465]:
vocab = sorted(vocab, reverse= True)
print(f'updated vocab: {vocab}')

updated vocab: ['soccer', 'socce', 'socc', 'soc', 'so', 'pl', 'is', '##y', '##ude', '##ud', '##u', '##t', '##s', '##r', '##o', '##ng', '##n', '##l', '##i', '##h', '##g', '##e', '##d', '##c', '##a']


In [466]:
string = 'playing'
result = []

while (i > 0):
    i = len(string)
    while string[:i] not in vocab:
        i -= 1
        if i == 0:
            break
    if string[:i] != '':
        result.append(string[:i])
    string = '##' + string[i:]

result

['pl', '##a', '##y', '##i', '##ng']